In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)
tf.test.is_gpu_available()
gpus = tf.config.list_physical_devices('GPU')
if gpus: 
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=14336 )] #7168  6144
    )

logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

In [ ]:
#Kütüphanelerin eklenmesi
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.model_selection import KFold, cross_val_predict, cross_val_score
!pip install utils
from utils import *
import random
import warnings
warnings.filterwarnings('ignore') 
#from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score, make_scorer,precision_score,recall_score,f1_score
import nltk
nltk.download('punkt')
from spacy.lang.tr import Turkish
from tqdm import tqdm
import string
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from keras.utils import np_utils
!pip install transformers
!pip install bert-tensorflow
from transformers import AutoModel, AutoTokenizer
import nltk 
import torch
import tqdm
import math
import bert
from tensorflow.keras import layers
from transformers import AutoModel, AutoTokenizer
import re
import random

# DEEP LEARNING IMPORTS
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, Dropout, Flatten, MaxPooling2D,Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from sklearn.pipeline import Pipeline

import math
import bert
from tensorflow.keras import layers
from transformers import AutoModel, AutoTokenizer
import re
import random

In [ ]:
column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_total.xlsx")

df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()

In [ ]:
df=df.sample(frac=1).reset_index(drop=True)
df

In [ ]:
tweets = list(df['tweets'])


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased",do_lower_case=True)

In [ ]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [ ]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2
DROPOUT_RATE = 0.2
NB_EPOCHS = 50
early_stopping = EarlyStopping(monitor='loss', patience=3, verbose=True)

In [ ]:
#rnn
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=256,
                 dnn_units=512,
                 model_output_classes=3,
                 dropout_rate=0.3,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.rnn_layer1 = layers.SimpleRNN(128,
                                        return_sequences= True,
                                        activation="relu")
        self.rnn_layer2 = layers.SimpleRNN(256,
                                        return_sequences= True,
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)

        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.rnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.rnn_layer2(l) 
        l_2 = self.pool(l_2)
        
        concatenated = tf.concat([l_1,l_2], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1,mcc,cohen_kappa=[],[],[],[],[],[]
BATCH_SIZE = 128

for train_index, test_index in kfold.split(df):
    # splitting Dataframe (dataset not included)
    
    train_df = df.iloc[train_index].tweets.tolist()
    test_df = df.iloc[test_index].tweets.tolist()
    train_y = np.array(df.iloc[train_index].duygu.tolist())
    test_y = np.array(df.iloc[test_index].duygu.tolist())

    tokenized_reviews_train = [tokenize_reviews(tweet) for tweet in train_df]
    tokenized_reviews_test = [tokenize_reviews(tweet) for tweet in test_df] 
    
    reviews_with_len_train = [[review, train_y[i], len(review)]
                     for i, review in enumerate(tokenized_reviews_train)]
    
    reviews_with_len_test = [[review, test_y[i], len(review)]
                     for i, review in enumerate(tokenized_reviews_test)]

    random.shuffle(reviews_with_len_train)
    random.shuffle(reviews_with_len_test)
    
    reviews_with_len_train.sort(key=lambda x: x[2])
    reviews_with_len_test.sort(key=lambda x: x[2])
    
    sorted_reviews_train = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len_train]
    sorted_reviews_test = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len_test]

    y_true = [review_lab[1] for review_lab in reviews_with_len_test]
    
    processed_dataset_test = tf.data.Dataset.from_generator(lambda: sorted_reviews_test, output_types=(tf.int32, tf.int32))
    processed_dataset_train = tf.data.Dataset.from_generator(lambda: sorted_reviews_train, output_types=(tf.int32, tf.int32))

    test_data = processed_dataset_test.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))
    train_data = processed_dataset_train.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))
    
    """ TOTAL_BATCHES = len(sorted_reviews_train)
    TEST_BATCHES = len(sorted_reviews_test)
    batched_dataset_train.shuffle(TOTAL_BATCHES)
    batched_dataset_test.shuffle(TEST_BATCHES)
    test_data = batched_dataset_test.take(TEST_BATCHES)
    train_data = batched_dataset_train.skip(TEST_BATCHES)"""

    text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

    opt = tf.keras.optimizers.SGD(learning_rate=0.05)

    if OUTPUT_CLASSES == 2:
        text_model.compile(loss="binary_crossentropy",
                          optimizer="adam",
                          metrics=["accuracy"])
    else:
        text_model.compile(loss="sparse_categorical_crossentropy",
                          optimizer=opt,
                          metrics=["sparse_categorical_accuracy"])

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    text_model.fit(train_data, epochs=NB_EPOCHS,callbacks=early_stopping)
    
    # evaluate
    loss, accuracy = text_model.evaluate(test_data)
    preds = text_model.predict(test_data)
    y_pred=preds.argmax(axis=1)
    
    precision= precision_score(y_true, y_pred, average='weighted')
    recall= recall_score(y_true, y_pred, average='weighted')
    f1_measure = f1_score(y_true, y_pred, average='weighted')
    mcc_score = matthews_corrcoef(y_true, y_pred)
    c_kappa=cohen_kappa_score(y_true, y_pred)
    
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_measure)
    mcc.append(mcc_score)
    cohen_kappa.append(c_kappa)

    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
def Average(lst):
    return sum(lst) / len(lst)

In [ ]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))
print('test mcc: %f' % (Average(mcc)))
print('test cohen_kappa: %f' % (Average(cohen_kappa)))